<a href="https://colab.research.google.com/github/smohanna/EAEE4000_Rainfall_Intensity/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Rode uma vez para garantir as libs instaladas
!pip install -q xee rioxarray earthengine-api geemap


In [2]:
# --- 1. IMPORTAÇÃO ---

import ee
import xarray as xr
import xee      # ⚠️ IMPORTANTE: só de importar ele registra o backend 'ee'
import pandas as pd
import numpy as np

print("Engines registrados no xarray:")
print(xr.backends.list_engines())



Engines registrados no xarray:
{'h5netcdf': <H5netcdfBackendEntrypoint>
  Open netCDF (.nc, .nc4 and .cdf) and most HDF5 files using h5netcdf in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.H5netcdfBackendEntrypoint.html, 'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .cdf and .nc.gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'ee': <EarthEngineBackendEntrypoint>, 'rasterio': <RasterioBackend>, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [3]:
# --- 2. AUTENTICAÇÃO ---
try:
    # Substitua 'pacific-song-279418' pelo seu projeto se for diferente
    ee.Initialize(project='pacific-song-279418')
    print("Google Earth Engine autenticado!")
except:
    print("Pedindo autenticação...")
    ee.Authenticate()
    ee.Initialize(project='pacific-song-279418')
    print("Autenticado com sucesso!")

Google Earth Engine autenticado!


In [4]:
# --- 3. DEFINIÇÃO DA COLEÇÃO (CHIRPS - MAIO 2024) ---
print(">>> Definindo coleção CHIRPS...")
var_name   = 'precipitation'
time_start = '2023-01-01'
time_end   = '2024-06-01'    # Pegando até Junho para garantir Maio completo

# Bounding Box do RS
xMin, yMin, xMax, yMax = -57.0, -34.0, -49.0, -27.0

collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
             .filterDate(time_start, time_end) \
             .filterBounds(ee.Geometry.Rectangle(xMin, yMin, xMax, yMax)) \
             .select(var_name)

>>> Definindo coleção CHIRPS...


In [5]:
# --- 4. BAIXAR O DATASET (XARRAY) ---
print(">>> Baixando dados para Xarray (Isso pode levar uns segundos)...")
ds = xr.open_dataset(
    collection,
    engine='ee',
    scale=0.1,  # CHIRPS é 0.05 ou 0.1 deg
    projection=collection.first().select(0).projection(),
    geometry=ee.Geometry.Rectangle(xMin, yMin, xMax, yMax),
).rename({'lon': 'x', 'lat': 'y'})

# Reordenar dimensões (Time, Y, X)
ds = ds.transpose('time', 'y', 'x')

print("-" * 30)
print("SUCESSO!")
print("Data Final no Dataset:", ds[var_name].time.max().values)
print("Shape do Dataset:", ds[var_name].shape)
print("-" * 30)


>>> Baixando dados para Xarray (Isso pode levar uns segundos)...
------------------------------
SUCESSO!
Data Final no Dataset: 2024-05-31T00:00:00.000000000
Shape do Dataset: (517, 141, 160)
------------------------------


In [7]:

# --- 5. EXTRAIR PONTO ESPECÍFICO (EX: PORTO ALEGRE/CANOAS) ---
# Coordenadas aproximadas de uma área afetada (Canoas/RS)
# x = -51.17, y = -29.92
target_lon = -51.2
target_lat = -30.0

print(f">>> Extraindo série temporal para o ponto ({target_lat}, {target_lon})...")
ds_point = ds[var_name].sel(x=target_lon, y=target_lat, method='nearest')
df_point = ds_point.to_dataframe().reset_index()

# Limpeza final
df = df_point.rename(columns={var_name: 'rain_mm'})
df['date'] = pd.to_datetime(df['time'])
df = df[['date', 'rain_mm', 'x', 'y']].sort_values('date')

# Filtrar só Maio de 2024 para você ver a enchente
maio_24 = df[(df['date'] >= '2024-05-01') & (df['date'] <= '2024-05-31')]

print("\nPrévia dos dados de Maio/2024 (Enchente):")
print(maio_24.head(10))

# Salvar CSV
from google.colab import drive
drive.mount('/content/drive')

df_point.to_csv("/content/drive/MyDrive/chuva_RS_chirps_maio2024.csv", index=False)
print("Salvo em /content/drive/MyDrive/chuva_RS_chirps_maio2024.csv")


>>> Extraindo série temporal para o ponto (-30.0, -51.2)...

Prévia dos dados de Maio/2024 (Enchente):
          date    rain_mm       x       y
486 2024-05-01  48.976650 -51.225 -29.975
487 2024-05-02  42.854568 -51.225 -29.975
488 2024-05-03   6.122081 -51.225 -29.975
489 2024-05-04  24.488325 -51.225 -29.975
490 2024-05-05   6.122081 -51.225 -29.975
491 2024-05-06   0.000000 -51.225 -29.975
492 2024-05-07   0.000000 -51.225 -29.975
493 2024-05-08   4.966668 -51.225 -29.975
494 2024-05-09   0.000000 -51.225 -29.975
495 2024-05-10   9.933336 -51.225 -29.975
Mounted at /content/drive
Salvo em /content/drive/MyDrive/chuva_RS_chirps_maio2024.csv
